<a href="https://colab.research.google.com/github/woo-jungnam/TH-TTNT/blob/main/Minimax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math

# =========================================================
# 1. CẤU HÌNH GAME
# =========================================================
class GameConfig:
    N = 10
    K = 5
    MAX_DEPTH = 5

    EMPTY = 0
    AI = 1
    HUMAN = -1

    DIRECTIONS = [
        (1, 0),
        (0, 1),
        (1, 1),
        (1, -1)
    ]


# =========================================================
# 2. BOARD
# =========================================================
class Board:
    def __init__(self, config: GameConfig):
        self.cfg = config
        self.grid = [[config.EMPTY for _ in range(config.N)]
                     for _ in range(config.N)]

    def print(self):
        symbols = {
            self.cfg.AI: "X",
            self.cfg.HUMAN: "O",
            self.cfg.EMPTY: "."
        }
        for row in self.grid:
            print(" ".join(symbols[c] for c in row))
        print()

    def place(self, x, y, player):
        self.grid[x][y] = player

    def remove(self, x, y):
        self.grid[x][y] = self.cfg.EMPTY

    def check_win(self, player):
        N, K = self.cfg.N, self.cfg.K

        for i in range(N):
            for j in range(N):
                if self.grid[i][j] != player:
                    continue

                for dx, dy in self.cfg.DIRECTIONS:
                    cnt = 0
                    x, y = i, j
                    while 0 <= x < N and 0 <= y < N and self.grid[x][y] == player:
                        cnt += 1
                        if cnt == K:
                            return True
                        x += dx
                        y += dy
        return False


# =========================================================
# 3. EVALUATOR (HEURISTIC) tính điểm
# =========================================================
class Evaluator:
    def __init__(self, board: Board):
        self.board = board
        self.cfg = board.cfg

    def evaluate(self):
        if self.board.check_win(self.cfg.AI):
            return 100000
        if self.board.check_win(self.cfg.HUMAN):
            return -100000

        score = 0
        for i in range(self.cfg.N):
            for j in range(self.cfg.N):
                if self.board.grid[i][j] != self.cfg.EMPTY:
                    for dx, dy in self.cfg.DIRECTIONS:
                        score += self.evaluate_line(i, j, dx, dy)
        return score

    def evaluate_line(self, x, y, dx, dy):
        player = self.board.grid[x][y]
        count = 0
        open_ends = 0

        i, j = x, y
        while 0 <= i < self.cfg.N and 0 <= j < self.cfg.N and self.board.grid[i][j] == player:
            count += 1
            i += dx
            j += dy

        if 0 <= i < self.cfg.N and 0 <= j < self.cfg.N and self.board.grid[i][j] == self.cfg.EMPTY:
            open_ends += 1

        i, j = x - dx, y - dy
        while 0 <= i < self.cfg.N and 0 <= j < self.cfg.N and self.board.grid[i][j] == player:
            count += 1
            i -= dx
            j -= dy

        if 0 <= i < self.cfg.N and 0 <= j < self.cfg.N and self.board.grid[i][j] == self.cfg.EMPTY:
            open_ends += 1

        if count >= self.cfg.K:
            return 10000 * player
        if count == 4 and open_ends:
            return 1000 * player
        if count == 3 and open_ends:
            return 200 * player
        if count == 2 and open_ends:
            return 50 * player

        return 0


# =========================================================
# 4. MOVE GENERATOR
# =========================================================
class MoveGenerator:
    def __init__(self, board: Board):
        self.board = board
        self.cfg = board.cfg

    def generate(self):
        moves = set()
        for i in range(self.cfg.N):
            for j in range(self.cfg.N):
                if self.board.grid[i][j] != self.cfg.EMPTY:
                    for dx in range(-1, 2):
                        for dy in range(-1, 2):
                            x, y = i + dx, j + dy
                            if 0 <= x < self.cfg.N and 0 <= y < self.cfg.N:
                                if self.board.grid[x][y] == self.cfg.EMPTY:
                                    moves.add((x, y))

        if not moves:
            return [(self.cfg.N // 2, self.cfg.N // 2)]
        return list(moves)


# =========================================================
# 5. MINIMAX AI
# =========================================================
class MinimaxAI:
    def __init__(self, board: Board):
        self.board = board
        self.cfg = board.cfg
        self.evaluator = Evaluator(board)
        self.generator = MoveGenerator(board)

    def minimax(self, depth, alpha, beta, is_max):
        if depth == 0 or self.board.check_win(self.cfg.AI) or self.board.check_win(self.cfg.HUMAN):
            return self.evaluator.evaluate(), None

        best_move = None

        if is_max:
            max_eval = -math.inf
            for x, y in self.generator.generate():
                self.board.place(x, y, self.cfg.AI)
                eval_score, _ = self.minimax(depth - 1, alpha, beta, False)
                self.board.remove(x, y)

                if eval_score > max_eval:
                    max_eval = eval_score
                    best_move = (x, y)

                alpha = max(alpha, eval_score)
                if beta <= alpha:
                    break
            return max_eval, best_move
        else:
            min_eval = math.inf
            for x, y in self.generator.generate():
                self.board.place(x, y, self.cfg.HUMAN)
                eval_score, _ = self.minimax(depth - 1, alpha, beta, True)
                self.board.remove(x, y)

                if eval_score < min_eval:
                    min_eval = eval_score
                    best_move = (x, y)

                beta = min(beta, eval_score)
                if beta <= alpha:
                    break
            return min_eval, best_move


# =========================================================
# 6. GAME LOOP
# =========================================================
class Game:
    def __init__(self):
        self.cfg = GameConfig()
        self.board = Board(self.cfg)
        self.ai = MinimaxAI(self.board)

    def run(self):
        print("=== BẮT ĐẦU GAME ===")
        self.board.print()

        while True:
            x, y = map(int, input("Nhập nước đi (x y): ").split())
            self.board.place(x, y, self.cfg.HUMAN)
            self.board.print()

            if self.board.check_win(self.cfg.HUMAN):
                print("BẠN THẮNG!")
                break

            print("AI đang suy nghĩ...")
            _, move = self.ai.minimax(
                self.cfg.MAX_DEPTH,
                -math.inf,
                math.inf,
                True
            )

            if move:
                self.board.place(move[0], move[1], self.cfg.AI)
            self.board.print()

            if self.board.check_win(self.cfg.AI):
                print("AI THẮNG!")
                break


# =========================================================
# 7. MAIN
# =========================================================
if __name__ == "__main__":
    Game().run()


=== BẮT ĐẦU GAME ===
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .

Nhập nước đi (x y): 0 0
O . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .

AI đang suy nghĩ...
O . . . . . . . . .
. X . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .

Nhập nước đi (x y): 0 1
O O . . . . . . . .
. X . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .

AI đang suy nghĩ...
O O . . . . . . . .
. X X . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . .